In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os
import requests
import zipfile


for ticker in ["BTCUSDC", "SANDUSDC", "XRPUSDC", "ETHUSDC"]:
    url = f"https://data.binance.vision/?prefix=data/spot/monthly/klines/{ticker}/30m/"
    output_csv = f"tmp/{ticker}.csv"
    target_csv = f"assets/{ticker}.csv"

    # Set up Selenium WebDriver
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    # service = Service('path/to/msedgedriver')  # Replace with the path to your Edge WebDriver
    driver = webdriver.Edge(options=options)

    try:
        # Launch the URL
        driver.get(url)
        time.sleep(5)  # Wait for 5 seconds

        # Find all links ending with ".zip"
        links = driver.find_elements(By.XPATH, "//a[substring(@href, string-length(@href) - 3) = '.zip']")
        zip_links = [link.get_attribute('href') for link in links]

        for link in zip_links:
            print(link)

        # Create the local folder if it doesn't exist
        os.makedirs('tmp', exist_ok=True)

        with open(output_csv, 'w') as file:
            file.write("Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore\n")

        # Download and save each linked resource
        for i, link in enumerate(zip_links):
            print(f"{i+1}/{len(zip_links)}")

            file_name = os.path.join('tmp', os.path.basename(link))
            response = requests.get(link)
            with open(file_name, 'wb') as file:
                file.write(response.content)

            time.sleep(1)

            if zipfile.is_zipfile(file_name):
                with zipfile.ZipFile(file_name, 'r') as zip_ref:
                    zip_ref.extractall('tmp')

            os.remove(file_name)

            time.sleep(1)

            # Get the name of the extracted CSV file
            extracted_files = zip_ref.namelist()
            if len(extracted_files) == 1:
                extracted_csv = os.path.join('tmp', extracted_files[0])
                
                # Append the content of the extracted CSV to master.csv
                with open(extracted_csv, 'r') as source_file:
                    with open(output_csv, 'a') as master_file:
                        master_file.write(source_file.read())
                
                # Delete the original extracted CSV file
                os.remove(extracted_csv)
            time.sleep(1)
        
        # Move the consolidated CSV to the target location
        os.makedirs(os.path.dirname(target_csv), exist_ok=True)
        os.replace(output_csv, target_csv)
    finally:
        driver.quit()

https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2025-02.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2025-01.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-12.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-11.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-10.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-09.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-08.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-07.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-06.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-05.zip
https://data.binance.vision/data/spot/monthly/klines/BTCUSDC/30m/BTCUSDC-30m-2024-04.zip
https://data.binance.